In [1]:
import torch
import pandas as pd
import sys
import os

In [2]:
sys.path.append('../src')
from preprocess import initialize_clip_model, generate_embedding
from retrieval import hybrid_retrieval, PostgresVectorRetrieval, TextSearchRetrieval, FaissVectorRetrieval

/Users/jchang/miniforge3/envs/finly/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
print(f"Device is {device}")

Device is mps


In [4]:
df = pd.read_csv('benchmark.csv')
df.head()

,Pid,Name,Query
0,230765.156074.66EA81FE0710E584.26DCA022C55D334...,TEMU 26pcs Silicone Daisy Beads Set,silicone daisy beads
1,230765.156074.66EA81FE0710E584.26DCA022C55D334...,TEMU 26pcs Silicone Daisy Beads Set,daisy bead making kit for keychains and bracelets
2,230765.156074.66EA81FE0710E584.26DCA022C55D334...,TEMU 26pcs Silicone Daisy Beads Set,TEMU 26pcs silicone daisy crafting beads set f...
3,178866.156074.820F1205554371C6.94435B3E5252BCD...,This Annoying Home Life: A Mindless Coloring B...,annoying life coloring book
4,178866.156074.820F1205554371C6.94435B3E5252BCD...,This Annoying Home Life: A Mindless Coloring B...,stress relief coloring book for adults


In [5]:
df = pd.read_parquet('../data/merged_output_sample_100k.parquet')
df = df.sample(n=10, random_state=42)
df['Query'] = df['Name']
df

,Pid,Name,ShortDescription,Description,CategoryId,Category,ImageURL,Price,PriceCurrency,SalePrice,...,Gender,Size,Condition,AlternateImageUrl,AlternateImageUrl2,AlternateImageUrl3,AlternateImageUrl4,DeepLinkURL,LinkUrl,Query
75721,202186.2.AA90356285D69126.84E21FFE69AF8690.80I...,Pantalones De Lana,,Peso de la tela: 480gr /m2. Cierre frontal ocu...,204.0,Apparel & Accessories >Clothing >Pants,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,2450.00,USD,2450.00,...,male,48,new,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://www.luisaviaroma.com/es-us/p/zegna/hom...,https://track.flexlinks.com/p.ashx?foc=101&fop...,Pantalones De Lana
80184,228963.156074.5BD1DE98FA94FFB1.9A534E9C206F812...,Adult adidas Adilette Comfort Slide Sandals Bl...,,Slide feet first into comfort. These adidas sl...,187.0,Apparel & Accessories >Shoes,https://scheels.scene7.com/is/image/Scheels/19...,35.00,USD,35.00,...,,M6/W7,New,https://scheels.scene7.com/is/image/Scheels/19...,https://scheels.scene7.com/is/image/Scheels/19...,https://scheels.scene7.com/is/image/Scheels/19...,,https://www.scheels.com/p/19573478513,https://track.flexlinks.com/p.ashx?foc=101&fop...,Adult adidas Adilette Comfort Slide Sandals Bl...
19864,159390.1.5EDD.93843A69CAB8D972.US-17042811QX-3...,Atlantic Stars Man Sneakers Black Size 6 Soft ...,"leather, techno fabric, suede effect, logo, mu...","leather, techno fabric, suede effect, logo, mu...",187.0,Apparel & Accessories >Shoes,https://cdn.yoox.biz/17/17042811QX_12_F.JPG,280.00,USD,184.00,...,male,6,,,,,,https://www.yoox.com/US/17042811QX/item?dept=men,https://track.flexlinks.com/p.ashx?foc=101&fop...,Atlantic Stars Man Sneakers Black Size 6 Soft ...
76699,202186.2.D25905FC54A031D8.20538B33EF063483.79I...,Polka Dot Silk Short Sleeve Long Dress,,Self-tie closure at neckline. Side zip closure...,2271.0,Kleding en accessoires >Kleding >Jurken,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,1345.00,EUR,941.00,...,female,36 - 46,new,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://www.luisaviaroma.com/en-nl/p/alessandr...,https://track.flexlinks.com/p.ashx?foc=101&fop...,Polka Dot Silk Short Sleeve Long Dress
92991,230765.156074.B2757A37DA3B9EB5.3EE61EC8BB28C48...,TEMU Conjunto de 10 peças de Adaptador de Chav...,,O envio mais rápido. O melhor serviço,1624.0,Comercial e industrial >Ciência e laboratório,https://img.kwcdn.com/product/fancy/1cd67cfe-8...,250.19,BRL,180.88,...,,,New,,,,,https://www.temu.com/br/goods.html?_bg_fs=1&_p...,https://track.flexlinks.com/p.ashx?foc=101&fop...,TEMU Conjunto de 10 peças de Adaptador de Chav...
76434,202186.2.CE3A812042ED7199.8E7A8E0A5D3B365F.79I...,Dina,,. .. 36,1581.0,Apparel & Accessories >Clothing >Skirts,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,1137000.00,KRW,568500.00,...,female,34 - 36,new,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://ximages.luisaviaroma.com/Zoom/c/2024/0...,https://www.luisaviaroma.com/ko-kr/p/isabel-ma...,https://track.flexlinks.com/p.ashx?foc=101&fop...,Dina
84004,230765.156074.36ECD19AEBC6E493.1933AD200E04DAD...,"TEMU & Key Linen Placemats - Set Of 4/6, 12.6""...",,Faster shipping. Better service,601.0,Home & Garden >Linens & Bedding >Table Linens,https://img.kwcdn.com/product/fancy/c094e1b8-4...,49.99,CAD,9.28,...,,,New,,,,,https://www.temu.com/ca/goods.html?_bg_fs=1&_p...,https://track.flexlinks.com/p.ashx?foc=101&fop...,"TEMU & Key Linen Placemats - Set Of 4/6, 12.6""..."
80917,228963.156074.5BD1DE98FA94FFB1.54B617D961A48A0...,Men's HOKA Clifton 9 Running Shoes 11 Regular ...,,The Men's HOKA Clifton 9 Running Shoes are the...,187.0,Ap

In [6]:
# Models to test
clip_model = "openai/clip-vit-base-patch32"
initialize_clip_model(clip_model)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


(CLIPProcessor:
 - image_processor: CLIPImageProcessor {
   "crop_size": {
     "height": 224,
     "width": 224
   },
   "do_center_crop": true,
   "do_convert_rgb": true,
   "do_normalize": true,
   "do_rescale": true,
   "do_resize": true,
   "image_mean": [
     0.48145466,
     0.4578275,
     0.40821073
   ],
   "image_processor_type": "CLIPImageProcessor",
   "image_std": [
     0.26862954,
     0.26130258,
     0.27577711
   ],
   "resample": 3,
   "rescale_factor": 0.00392156862745098,
   "size": {
     "shortest_edge": 224
   }
 }
 
 - tokenizer: CLIPTokenizerFast(name_or_path='openai/clip-vit-base-patch32', vocab_size=49408, model_max_length=77, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
 	49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_wo

In [8]:
# Database configuration
DB_CONFIG = {
    'dbname': os.getenv('PGDATABASE', 'finly'),
    'user': os.getenv('PGUSER', 'postgres'),
    'password': os.getenv('PGPASSWORD', 'postgres'),
    'host': os.getenv('PGHOST', 'localhost'),
    'port': os.getenv('PGPORT', '5432')
}
TOP_K = 5

# Create components
components = [
    #PostgresVectorRetrieval('text_embedding', DB_CONFIG),
    FaissVectorRetrieval(index_type='text'),
    PostgresVectorRetrieval('image_embedding', DB_CONFIG),
    TextSearchRetrieval('ts_rank_cd', DB_CONFIG)
]

weights = [1, 0, 0]  # Must sum to 1

# Initialize counters
hits = 0
total = len(df)

# Process each query individually
for i, row in df.iterrows():
    query = row['Query']
    query_embedding = generate_embedding(query_text=query)
    
    # Run hybrid search for single query
    pids, scores = hybrid_retrieval(
        query=query,
        query_embedding=query_embedding,
        components=components,
        weights=weights,
        top_k=TOP_K
    )
    
    # Check if the ground truth Pid is in the results
    if row['Pid'] in pids:
        hits += 1

recall_at_k = hits / total
print(f"Recall@{TOP_K}: {recall_at_k:.4f}")

Recall@5: 0.0000


In [9]:
query = generate_embedding("Polka Dot Silk Short Sleeve Long Dress")

# 1. Query score from FAISS
scores_faiss = FaissVectorRetrieval(index_type='text').score(query)

# 2. Query score from PostgreSQL
scores_pg = PostgresVectorRetrieval('text_embedding', DB_CONFIG).score(query)

# Compare top results
print("Top FAISS:", list(scores_faiss.items())[:5])
print("Top PG:", list(scores_pg.items())[:5])

AttributeError: 'str' object has no attribute 'reshape'